# 1. Introducción
Vamos a hacer un estudio sobre la base de datos *Suicide Rates Overview 1985 to 2016*, que es un conjunto compilado de otras 4 fuentes de informació, *United Nations Development Program*, *World Bank*, un dataset de *kaggle* denominado *Suicide in the Twenty-First Century* y la *World Health Organization*. El conjunto de estos datos forma esta base de datos que contiene el ratio de suicidios por país y año dividido por lo diferentes grupos que hay en la ciudad, grupos distingidos por sexo y franja de edad. Si bien no hay un objetivo especificado la motivación de este conjunto de datos es la prevención de los suicidios, es por esto que vamos a afrontar con diferentes enfoques para ver si se puede realizar un estudio que nos ayude a predecir las causas de estos sucesos.

# Librerias
Las diferentes librerías que se han necesitado se encuentran aquí.

In [ ]:
from sklearn.datasets import make_regression
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import category_encoders as ce #pip install category_encoders
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
import pandas_profiling as pdp #conda install -c conda-forge pandas-profiling

# 2. Exploratory Data Analysis
Lo primero que se va a hacer es cargar los datos en un objeto *dataframe* de la librería *pandas* para poder hacer el EDA y eliminar aquellas columnas que aporten información redundante. Identificaremos que atributos contiene la base de datos y si tiene sentido tenerlos en cuenta en nuestro estudio.

En esta función cargamos los datos en un objeto dataframe de la libreria de pandas

In [ ]:
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

Con esta función eliminamos las columnas que tengan más de un 65% de los datos perdidos, y para los que tienen menos de un 65% eliminamos toda la fila

In [ ]:
def missing_values(dataset):
    nanNum = dataset.isnull().sum()
    print(nanNum/dataset.shape[0]*100)
    #Si columna con 30% de valores missing la eliminamos
    for (index, item) in enumerate(nanNum):
        if item > dataset.shape[0]*0.65:
            dataset = dataset.drop(dataset.columns[index], axis=1)
            
    #Si hay algún missing value eliminamos la fila
    dataset = dataset.dropna(axis=0)
    
    return dataset

Aplicando la técnica de *Z-Score* identificaremos los valores que son demasiado distantes de la distribución y los eliminaremos considerandolos *outliers*.

In [ ]:
def remove_outliers(df, df_entero):
    z_scores = scipy.stats.zscore(df)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    return df_entero[filtered_entries]

Aqui cargamos los datos y observamos como son nuestros datos

In [ ]:
path = '../input/suicide-rates-overview-1985-to-2016/master.csv'
dataset = load_dataset(path)
print('Dimensión inicial del dataset es de ', dataset.shape)
dataset.head()

Como podemos ver estos datos van a tener que adaptarse para poder trabajar con ellos ya que existen muchos valores discretos que no podemos tratar tal y como estan. También vemos que la columna *HDI for year* tiene los 5 primeros valores a *NaN*, esto nos hace sospechar así que mirarmemos si son solo estos o esta columna tiene demasiados valores así.

Antes de continuar vamos a describir todos los atributos que contienen nuestros datos de manera que cuando avancemos y tomemos ciertas decisiones se entienda bien lo que se esta haciendo:

- *country*: País donde ha ocurrido.

- *year*: Año que ha sucedido.

- *sex*: Genero del fallecido.

- *age*: Edad del fallecido.

- *suicides_no*: Número de suicidios.

- *population*: Población.

- *suicides/100k pop*: Proporción del número de suicidios por cada 100 mil personas.

- *country-year*: País y año del suceso.

- *HDI for year: Índice de desarrollo humano, es una media aritmética de tres indices, tener una vida larga y saludable, adquirir conocimientos y disfrutar de un nivel de vida digno.
- gdp_for_year (\$)*: PIB por año.

- *gdp_per_capita (\$)*: PIB per cápita.

- *generation*: Generación del fallecido.

Ahora que ya tenemos una idea de como es nuestro *dataframe* vamos a ver el report de los datos, analizaremos lo más importante para poder tomar posteriores decisiones de manera correcta.

In [ ]:
prof = pdp.ProfileReport(dataset)
prof.to_file(output_file='report.html')
prof

Lo primero que vamos a mirar son los missing values, en este caso ningún atributo tiene, excepto la columna *HDI for year* que contiene el 69.9% de valores *NaN*. Para solucionar esto vamos a aplicar la función que habíamos creado antes que si una columna tenía más de 65% de valores perdidos se eliminaba directamente.

Lo segundo que vemos es que los atributos *country*, *country-year* y *gdp_for_year (\$)* tienen una alta cardinalidad, pero no hay que preocuparse, ya que el último es simplemente que el formato está mal, debería ser de tipo numérico pero está representado como si fuese un *string*. El caso del *country-year* es que es un atributo de la combinación de *country* y de *year* así que acabará siendo eliminado.

Los atributos *age*, *sex*, *country-year* y *gdp_for_year (\$)* están balanceados lo cuál es bueno, ya que esto va hacer que no se distorsionen los cálculos.

Las columnas *suicides_no* y *suicides/100k pop* tiene muchos valores que son 0, concretamente el 15.4%, el atributo *suicides_no* no importa porque será eliminado, ya que mirar el número de suicidios en total es bastante deficiente y nos basaremos en la proporción por cada 100 mil personas, pero ese atributo también tiene muchos 0, es algo que de momento obviaremos y más adelante lo retomaremos para darle una solución.

Mirando el mapa de correlaciones podemos ver que excepto el número de suicidios con la población no hay atributos muy correlacionados, pero esto es lo que comentabamos antes de porque vamos a eliminar la columna *suicides_no*. Otra cosa que no se puede apreciar en el mapa de correlaciones es que el PIB per cápita y el de por año siguen una correlación de 1, ya que son proporcionales, así que eliminaremos el *gdp_for_year (\$)*, ya que al no estar en formato numérico habría que transformarlo y es más simple utilizar el otro. La generación también se puede eliminar ya que tiene una alta relación con el atributo de la franja de edad, nos viene a explicar prácticamente lo mismo, además sabiendo el año y la franja de edad puedes saber la generación, realmente no es información adicional.

Lo que también se puede observar son los valores extremos de cada atributo, los atributos que tenemos que mirar a fondo para ver si hay *outliers* son los numéricos. El año no posee outliers ya que se mueve entre valores comprendidos entre 1985 y 2016, tal y como se especifica en el título de la página de la base de datos de *kaggle*. Los atributos que hemos dicho que vamos a eliminar no vamos a analizarlos. La población si que posee algunos *outliers* ya que el tercr percentil es de 1486143.25 y el percentil 95 es de 8850239.6 y si nos vamos al valor máximo es de 	43805214, estas diferencias entre los percentiles son demasiado grandes. El caso de *suicides/100k pop* también es grande, en el tercer percentil es de 16.62 y en el percentil 95 es de 50.5305 y el valor máximo asciende a 224.97. El *gdp_per_capita (\$)* también tiene algunos, ya que del percentil 95 que es de 54294 asciendo el valor máximo a 126352.

Antes de modificar las tablas vamos a analizar si estos outliers son anomalias o es por la descompensación entre paises, para hacer esto lo que vamos a hacer es ver de donde salen estos datos.

In [ ]:
dataset.sort_values(by=['population'], ascending=False).head(20)

Aquí parece que la descompensación esta en la población de estados unidos que es muy grande y por lo tanto distorsiona los datos, pero en este caso es mejor no eliminar estas filas ya que lo que haremos es dejar de tener en cuenta a la población de Estados Unidos y no sabiendo eso no podemos decir que aplique como estudio universal.

In [ ]:
dataset.sort_values(by=['suicides/100k pop'], ascending=False).head(20)

En el caso del ratio de suicidios si que son casos donde mejor no contarlos en la estadistica, ya son paises de población muy baja y que se han suicidado muy pocas personas, pero hace que la proporcion sea muy alta, y además no es siempre el mismo país por lo que parece que no es una tendencia de ese país si con son situaciones atípicas. Aunque si lo que vemos relevante es que la majoría de sitios que ocurre son en la Republica de Korea, así que aunque los vayamos a tratar como valores atípocos hay que tene en cuenta que no todos lo son.

In [ ]:
dataset.sort_values(by=['gdp_per_capita ($)'], ascending=False).head(20)

Lo primero que vemos es que Luxemburgo es país muy rico, pero si nos fijamos es siempre del mismo año, vamos a analizar si para este país son valores normales, si lo son habrá que dejarlos, pero si no lo son habrá que eliminarlos.

Después de ver esto concluimos que solo tenemos que eliminar los *outliers* de la columna *suicides/100k pop*.

In [ ]:
dataset[dataset['country'] == 'Luxembourg']['gdp_per_capita ($)'].describe()

Parece ser que este es un país bastante rico, por lo que tampoco deberiamos eliminar los *outliers*.

In [ ]:
dataset2 = missing_values(dataset)
dataset2.head()

Podemos ver que lo que deciamos anteriormente se corresponde con la ejecución de nuestra función y por lo tanto la columna *HDI for year* ha sido eliminada. El siguiente paso es eliminar aquellas columnas que anteriormente hemos dicho que no nos eran útiles porque estan formadas a través de la composición de otras o es información redundante que indice a errores a los algoritmos.

In [ ]:
dataset3 = dataset2.drop(['suicides_no', ' gdp_for_year ($) ', 'country-year', 'generation'], axis=1)
dataset3.head()

Aquí podemos ver como queda nuestro dataset después de la eliminación. El siguiente paso es la eliminación de los outliers de nuestros atributos numéricos, excepto del atributo *year*, tal y como habíamos dicho anteriormente. Vamos a aplicar el método que hemos creado al principio que aplica la técnica *Z-Score*

In [ ]:
dataset4 = remove_outliers(dataset3[['suicides/100k pop']], dataset3)
dataset4.describe()

Vemos que varias filas han sido eliminadas y las desviaciones de los datos ya no son tan grandes y los valores máximos son más reducidos ahora.

In [ ]:
print('Dimensiones del dataset limpio: ', dataset4.shape)
dataset4.head()

Podemos ver que se han eliminado alrededor de 800 elementos, es bastante poco teniendo en cuenta la cantidad de datos que tenemos, así que seguimos contando un buen volumen de datos para hacer el estudio.

# 3. Experimentación
Primero de todo sacaremos los histogramas para ver como están distribuidos los valores de los atributos.

In [ ]:
def histogramas(dataset, index):
    figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    data = dataset.values
    columnName = list(dataset)
    x = data[:, index]
    plt.figure()
    title="Histograma de l'atribut "
    plt.title(title)
    plt.xlabel(columnName[index])
    plt.ylabel("Count")
    hist = plt.hist(x, bins=11, histtype="bar", rwidth=0.8)
    plt.show()

In [ ]:
for i in range(dataset4.shape[1]):
    histogramas(dataset4, i)

Lo que queremos ver en este estudio es la proporción de suicidios por cada 100k habitantes. La distribución de estos es bastante malo por lo que la mejor es descomponer el estudio en otros de tal forma que podamos predecir algo. Se realizarían los siguientes estudios y luego se contrastaría la información entre estos para razonar cosas. Los estudios serían:

- Por cada país mirariamos los suicidios independientemente. Aquí podriamos ver más en detalle por cada país que ocurre, si una franja de edad o un sexo es más propenso.

- Por cada país y año mirariamos los suicidios totales, con este estudio podríamos ver la diferenca entre los paises y ver si paises ricos tiene más suicidios o al revés o si todos actuán de un forma similar.

- Por último haríamos una clusterización para agrupar los paises y así analizar ciertas tendencias seguún las características de cada grupo.

## 3.1. Estudio por paises
En este caso crearemos modelos predictivos para cada país para poder ver como actúa la población de un mismo lugar.

### 3.1.1 Preprocesado
Lo primero que vamos a hacer es crear un diccionario donda cada entrada será un dataframe con la información de cada país. Si un país tiene menos de 50 columnas es eliminado, ya que con pocos datos no se puede hacer nada.

De estos datos tenemos que eliminar la columna de *country* ya que ha dejado de tener sentido, ya que ahora los paises serán modelos independientes. Luego de esto vamos a mirar aquellos lugares que tengan en la mediana almenos un valor de 5, ya que con menos significa que casi todo es 0 y nos sirve de poco hacer un modelo sobre un país así.

In [ ]:
dict_df = {}
country_name = []
for item in dataset4['country'].unique():
    df = dataset4.loc[dataset4['country'] == item]
    if df.shape[0] > 50:
            dict_df[item] = df
            country_name.append(item)

In [ ]:
if 'country' in dict_df['United States']:
    for item in country_name:
        dict_df[item] = dict_df[item].drop(['country'], axis=1)
dict_df['United States'].head()

Aquí vemos un ejemplo de Estados Unidos de como quedan los dataset después de la eliminación de las columnas.

In [ ]:
aux = []
for item in country_name:
    if dict_df[item]['suicides/100k pop'].quantile(0.50) < 5:
        del dict_df[item]
        aux.append(item)
for x in aux:
    country_name.remove(x)
print('Número de países: ', len(country_name))
print(country_name)

En esta ejecución podemos ver que el número de países de los que se va hacer el estudio es de 53 y podemos ver el nombre de los países en cuestión. Ahora vamos a ver el histograma de los atributos para ver si han mejorado en algo.

In [ ]:
for item in country_name:
    print('----------------------------------------')
    print(item)
    for i in range(dict_df[item].shape[1]):
        histogramas(dict_df[item], i)

Siguen sin tener la mejor forma posible pero vemos que han mejorado bastante al separarlo por paises, ahora buscaremos las correlaciones entre los atributos y para eso lo que haremos será codificar el atributo *sex*, como 1 para *male* y 0 para *female* y también codificaremos el atributo *age* de manera ordinal, ya que al hacer eferencia a edades es lógico que por ejemplo el intervalo entre 55 y 74 años tenga un valor mayor al intervalo de 15 y 24 años.

In [ ]:
if dict_df['United States']['sex'].dtype != 'int64':
    le_s = preprocessing.LabelEncoder()
    le_a = preprocessing.LabelEncoder()
    age = np.sort(dataset3['age'].unique())
    for i in range(4):
        aux = age[i]
        age[i] = age[3]
        age[3] = aux
    le_s.fit(['female', 'male'])
    le_a.classes_ = age
    for item in country_name:
        dict_df[item][['sex']] = le_s.transform(dict_df[item]['sex'])
        dict_df[item][['age']] = le_a.transform(dict_df[item]['age'])
dict_df['United States'].head()

Podemos ver como se han quedado las columnas categoricas codificadas, con esto podemos sacar las correlaciones y observar los atributos que más relacionados están.

In [ ]:
for item in country_name:
    corr = dict_df[item].corr()

    plt.figure(figsize=(10,10))

    ax = sns.heatmap(corr, annot=True, linewidths=1, center=0)
    
    ax.set_title(item)


Entre cada país hay variaciones, pero por lo general vemos tendencia a ver los atributos de sexo y edad como los más relacionados, lo que implica que al ser de forma positiva existe cierta tendencia de ser el género masculino el que sufre de más suicidios y en una edad más avanzada, no son los más jovenes los que se suicidan más.

Los siguientes pasos son partir el dataset en test y train y estandatizarlo, para eso utilizaremos la función de abajo. Aplicaremos la estandarización *MinMaxScaler* ya que una estandarización estándar no daría bueno resultados, nuestros datos distan demasiado de una distribución normal así que que el cálculo daría unos malos resultados.

In [ ]:
def standarize(x_train, x_test):
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(x_train)
    return scaler.transform(x_train), scaler.transform(x_test)

In [ ]:
dict_x_y = {}
for item in country_name:
    dict_x_y[item] = {}
    dict_x_y[item]['x'] = dict_df[item].loc[:,dict_df[item].columns != "suicides/100k pop"].to_numpy()
    dict_x_y[item]['y'] = dict_df[item].loc[:,"suicides/100k pop"].to_numpy()
    dict_x_y[item]['x_s'], aux = standarize(dict_x_y[item]['x'], dict_x_y[item]['x'])
print(dict_x_y['United States']['x_s'][2])

Ahora tenemos un diccionario donde tenemos las x de estandarizadas y las y de cada país. En la ejecución podemos ver un ejemplo de una fila de atributos estandarizados de Estados Unidos.

Ya lo tenemos todo listo para poder realizar las experimentaciones así que en el siguiente subapartado procederemos a realizarlo.

### 3.1.2 Experimentación
Lo que vamos hacer en este apartado es hacer un cross validate para cada país aplicando una regresión lineal, sacaremos cual es el mejor modelo para diferentes k teniendo en cuenta la mse y el r2.

In [ ]:
country_scores = {}
for item in country_name:
    country_scores[item] = {}
    for i in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
        reg = LinearRegression()
        country_scores[item][i] = cross_validate(reg, dict_x_y[item]['x_s'], dict_x_y[item]['y'], cv=i, scoring=['neg_mean_squared_error', 'r2'], return_estimator=True)


In [ ]:
dict_scores_resume = {}
for item in country_name:
    dict_scores_resume[item] = {}
    for k in country_scores[item]:
        dict_scores_resume[item][k] = {}
        for numbers in country_scores[item][k]:
            i = 0
            aux = 0
            if numbers != 'estimator':
                dict_scores_resume[item][k][numbers] = {}
                for num in np.nditer(country_scores[item][k][numbers]):
                    i = i + 1
                    aux = aux + num
                result = aux/i
                dict_scores_resume[item][k][numbers] = result

In [ ]:
for pais, info_pais in dict_scores_resume.items():
    print(pais, ':')
    for k, info_k in info_pais.items():
        print('k =', k, ':')
        print('MSE:', info_k['test_neg_mean_squared_error'], 'R2:', info_k['test_r2'])
    print('--------------------------------------------------------------')

Podemos ver para cada país el mse y el r2 medio para cada k, hay que decir que el mse está negativo porque así lo fuerza la librería, con esto nos quieren indicar que esa variable es una variable a minimizar. Ahora lo que haremos será valorar cual es la mejor k para cada país y extraer el mejor modelo que haya creado la cross validación para poder ver los coeficientes y así ver a que atributos asigna más peso. 

In [ ]:
dict_best_k = {}
for pais, info_pais in dict_scores_resume.items():
    dict_best_k[pais] = {}
    dict_best_k[pais]['k'] = -1
    dict_best_k[pais]['value'] = float('-inf')
    for k, values in info_pais.items():
        if dict_best_k[pais]['value'] < (values['test_neg_mean_squared_error'] + values['test_r2'])/2:
            dict_best_k[pais]['value'] = (values['test_neg_mean_squared_error'] + values['test_r2'])/2
            dict_best_k[pais]['k'] = k
    val = country_scores[pais][dict_best_k[pais]['k']]['test_neg_mean_squared_error'] + country_scores[pais][dict_best_k[pais]['k']]['test_neg_mean_squared_error']
    dict_best_k[pais]['iter'] = val.argmax()
print(dict_best_k['Spain'])

Aquí lo que hemos extraido es la mejor k con el valor medio entre el valor del mse y el del r2. Dentro de esa k el mejor modelo encontrado sería el sexto en caso de España, esto nos servirá para encontrar el mejor regresor.

In [ ]:
dict_best_regr = {}
for item in country_name:
    dict_best_regr[item] = country_scores[item][dict_best_k[item]['k']]['estimator'][dict_best_k[item]['iter']]
print(dict_best_regr['Spain'])

Aquí tenemos un diccionario con el mejor regresor lineal por cada país, como se ve en la ejecución lo único que tenemos es el objeto del regresor lineal que nos servirá para llegar a conclusiones.

### 3.1.3. Conclusiones
Una vez ya hemos encontrado el mejor modelo vamos a enseñar para cada país que pesos a asignado a cada atributos y analizaremos un poco algunos de los países más interesantes, como son España, Estados Unidos, Francia.

In [ ]:
for pais, info_pais in dict_scores_resume.items():
    print(pais, ':')
    print('k =', dict_best_k[pais]['k'], ':')
    print('MSE:', country_scores[pais][dict_best_k[pais]['k']]['test_neg_mean_squared_error'][dict_best_k[pais]['iter']], 'R2:', country_scores[pais][dict_best_k[pais]['k']]['test_r2'][dict_best_k[pais]['iter']])
    print('--------------------------------------------------------------')

Aquí tenemos el error y el r2 para el mejor modelo de cada país, depende del país cambian mucho los valores, para algunos paises ha sido más fácil hacer la predicción y para otros ha sido imposible predecirlos. Ahora vamos a ver el peso que le han dado a caa uno de los atributos en los regresores de cada país y así podremos ver los atributos más importantes según el lugar.

In [ ]:
list_attr = ['country'] + list(dict_df['Spain'])
list_attr.remove('suicides/100k pop')
df_coef = pd.DataFrame(columns=list_attr)
i = 0
for pais, regr in dict_best_regr.items():
    df_coef.loc[i, 'country'] = pais
    df_coef.values[i, 1:] = regr.coef_
    i += 1
df_coef = df_coef.set_index('country')
df_coef.head(df_coef.shape[0])

En esta ejecución podemos ver para cada país que peso se ha asignado a cada uno de los atributos, se puede ver el país que se desee pero lo que vamos hacer es profundizar en tres países, España, Estados Unidos y Francia.

In [ ]:
df_coef[df_coef.index.isin(['Spain', 'United States', 'France'])].plot(kind='bar', figsize=[15,10],
             title ='Agrupación de paises por la proporción de suicidios en 2014')

Empezando por Francia podemos ver que la edad es el atributo con mayor pero con diferencia, seguido del sexo y luego los años de manera negativa, la población le otorga muy poca importancia para saber el ratio de suicidios.

En cuanto a España vemos que la edad también es el atributo más relevante, pero muy seguido de la población con signo negativo, luego tenemos el género también como importante, en este caso los menos relevante son los años, prácticamente tienen valor 0.

En Estados Unidos tenemos la edad y luego el género con valores muy parejos, el resto de atributos tiene poco valor, resaltando la riqueza del país como el menos importante.

De estos tres países podemos ver que actúan de una manera muy parecida, la edad y el género cobran mucha importancia a la hora de saber la proporción de suicidios.

## 3.2. Estudio conjunto del país
La idea de aquí es agrupar los suicidios de cada país por año y país sin distinguir por sexo ni edad, así podemos crear un modelo predictivo más general de lo que ocurre a nivel global

### 3.2.1 Preprocesado

Primeramente eliminamos los atributos de sexo, edad y generación y hacemos una agrupación por país y año. Luego de esto mostraremos los atributos a ver como quedan las distribuciones de los datos, y seguidamente mostraremos las correlaciones. La variable categorica del país no es ordinal por lo que no podemos hacer esto para ver su relación, mostraremos su relación con el atributo objetivo con un boxplot y después la codificaremos de forma binaria, ya que para una codificación *One Hot* tiene demasiada cardinalidad. Finalmente separaremos las muestras en entreno y validación y estandarizaremos con el *MinMaxScaler*, para la creación del set de entreno y el de validación lo que vamos hacer es entrenar con los datos hasta 2010 y validaremos con los años restante, así veremos si creando un regesor lineal de los años pasados podemos predecir lo que ha ocurrido en los últimos años.

In [ ]:
df = pd.DataFrame(columns=['country', 'year', 'population', 'suicides/100k pop', 'gdp_per_capita ($)'])
df['country'] = dataset4['country']
df['year'] = dataset4['year']
df['population'] = dataset4['population']
for i, fila in dataset4.iterrows():
    df.loc[i, 'suicides/100k pop'] = dataset2.loc[i, 'suicides_no']
    
df = df.fillna(0)
df = df.groupby(['country', 'year'], as_index=False).sum()

df_rich = dataset4.groupby(['country', 'year'], as_index=False).mean()
df['gdp_per_capita ($)'] = df_rich['gdp_per_capita ($)']
for i, fila in df.iterrows():
    df.loc[i, 'suicides/100k pop'] = (fila['suicides/100k pop'] / fila['population'])*100000
    
print(df.shape)
df.head()

Este es el resultado de la anteriormente descrito, podemos ver que nos hemos quedado con 2321 filas y solamente 5 columnas. Lo siguiente que haremos será printar los histogramas de los atributos para ver como se distribuyen los datos.

In [ ]:
for i in range(df.shape[1]):
    histogramas(df, i)

Podemos ver una clara mejora de los datos en el atributo objetivo, no se puede decir que posea distribución gaussiana pero aún así ha mejorado mucho su distribución.

Visto esto pasaremos a mirar las correlaciones entre atributos.

In [ ]:
corr = df.corr()

plt.figure(figsize=(10,10))

ax = sns.heatmap(corr, annot=True, linewidths=1)

Ni los años ni el PIB per cápita parecen tener relación con el atributo objetivo, esto nos da que pensar que lo que determina el número de suicidios es la localización con bastante independencia de su nivel económico.

Veremos como se relaciona el atributo *country* con un *boxplot*.

In [ ]:
plt.figure(figsize=(30, 100))
order = df.groupby(by=["country"])["suicides/100k pop"].mean().sort_values().iloc[::-1].index
ax = sns.boxplot(y="country", x="suicides/100k pop", data=df, order = order)

Pdemos ver que hay mucha variación dependiendo de cada país, esto nos hace pensar que este artibuto si que esta altamente correlacionado con el target, pero será el modelo quien nos determine esto exactamente.

Lo siguiente a hacer es aplicar la codificación para la variable *country*

In [ ]:
encoder = ce.binary.BinaryEncoder(cols=['country'])
df_e = encoder.fit_transform(df)
df_e = df_e.drop('country_0', axis=1)
df_e.head()

Aquí podemos ver como queda nuestro dataset ahora tenemos 7 columnas adicionales y hemos perdido la que nos indicaba el nombre del país con un *string*. Ahora pasaremos partir los datos en entrenamiento y validación como hemos dicho anteriormente y estandarizaremos con un *MinMaxScaler* ya que nuestros datos no tienen distribución normal y por lo tanto un *StandardScaler* daría malos resultados.

In [ ]:
x = df_e.loc[:,df_e.columns != "suicides/100k pop"].to_numpy()
y = df_e.loc[:,"suicides/100k pop"].to_numpy()

x_train = x[df_e.loc[:, 'year'] <= 2010]
x_val = x[df_e.loc[:, 'year'] > 2010]
y_train = y[df_e.loc[:, 'year'] <= 2010]
y_val = y[df_e.loc[:, 'year'] > 2010]

In [ ]:
x_train_s, x_val_s = standarize(x_train, x_val)
print(x_train_s[0])

Podemos ver como han quedado nuestras variables independientes estandarizadas, no pasa nada por haber estandarizado las columnas de country porque tal y como son no modican su valor y siguen siendo 1 y 0.

Ya tenemos todo preparado para comenzar a aplicar la regressión lineal, así que hasta aquí llega el preprocesado.

### 3.2.2. Experimentación
Lo que haremos aquí es aplicar un regresor lineal de grado 1 y de grado 2 en un cross validate con diferentes k para comprobar cual es la mejor manera de entrenar los datos, extraeremos el mejor modelo y luego haremos la validación con los datos de 2010 hasta 2016. Con esto veremos si es buen modelo predictivo o no.

In [ ]:
scores = {}
for d in [1, 2]:
    scores[d] = {}
    for i in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
        reg = LinearRegression()
        pf = PolynomialFeatures(degree=d)
        atribut_train=pf.fit_transform(x_train_s)
        scores[d][i] = cross_validate(reg, atribut_train, y_train, cv=i, scoring=['neg_mean_squared_error', 'r2'], return_estimator=True)


In [ ]:
scores_resume = {}
for k in scores:
    scores_resume[k] = {}
    for degree in scores[k]:
        scores_resume[k][degree] = {}
        for numbers in scores[k][degree]:
            i = 0
            aux = 0
            if numbers != 'estimator':
                scores_resume[k][degree][numbers] = {}
                for num in np.nditer(scores[k][degree][numbers]):
                    i = i + 1
                    aux = aux + num
                result = aux/i
                scores_resume[k][degree][numbers] = result

In [ ]:
for degree, info_degree in scores_resume.items():
    print('Regresion lineal de grado', degree, ':')
    for k, info_k in info_degree.items():
        print('k =', k, ':')
        print('MSE:', info_k['test_neg_mean_squared_error'], 'R2:', info_k['test_r2'])
    print('--------------------------------------------------------------')

Aquí podemos ver la media por cada k del mse y el r2, ahora con esta información pasaremos a encontrar el mejor regresor.

In [ ]:
best_k = {}
best_k['degree'] = -1
best_k['k'] = -1
best_k['value'] = float('-inf')
for degree, info_degree in scores_resume.items():
    for k, values in info_degree.items():
        if best_k['value'] < (values['test_neg_mean_squared_error'] + values['test_r2'])/2:
            best_k['value'] = (values['test_neg_mean_squared_error'] + values['test_r2'])/2
            best_k['k'] = k
            best_k['degree'] = degree
val = scores[best_k['degree']][best_k['k']]['test_neg_mean_squared_error'] + scores[best_k['degree']][best_k['k']]['test_neg_mean_squared_error']
best_k['iter'] = val.argmax()
print(best_k)

Aquí ya podemos ver cual es el mejor grado para la regresion, la mejor k y dentro de la cross validación la mejor iteración. Vamos ahora a utilizar ese regresor para hacer la validación con los últimos años.

In [ ]:
best_regr = scores[best_k['degree']][best_k['k']]['estimator'][best_k['iter']]
y_predict = best_regr.predict(np.insert(x_val_s, 0, 1, axis=1))
mse = mean_squared_error(y_val, y_predict)
r2 = r2_score(y_val, y_predict)
print('MSE:', mse, 'R2:', r2)

Tenemos el error y el coeficiente r2, con esto en el apartado siguiente llegaremos a las conclusiones pertinentes.

### 3.2.3 Conclusiones


Vemos que el error es bastante grande y el r2 es cercano a cero y negativo, lo cual indica que este no es un buen modelo, igualemente vamos a ver una gráfica que nos muestre los que a pasado desde 2011 hasta 2016 y que el lo que ha predicho nuestro modelo

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figheight(5)
fig.set_figwidth(15)

ax1.set(ylim=(0, 30))
ax1.set_title('Sucesos reales')
ax1.scatter(x_val[:, list(df_e).index('year')], y_val)

ax2.set(ylim=(0, 30))
ax2.set_title('Sucesos predecidos')
ax2.scatter(x_val[:, list(df_e).index('year')], y_predict, color='red')


En esta gráfica lo que podemos observar esque las predicciones de cada año no se corresponden demasiado con los sucesos que han pasado realmente, las predicciones por cada año son muy parecidas entre ellas. Por todo esto estimamos que las predicciones de estos últimos 6 años son malos y aproximadamente siempre nos devuelve el mismo valor. Ahora veremos una gráfica que nos muestra el error de cada muestra divididas por años. 

In [ ]:
y_plot = (y_val-y_predict)
plt.scatter(x_val[:, list(df_e).index('year')], y_plot)
plt.axhline(y=0, color='red', linewidth=2)

En esta gráfica podemos ver el error que se cometido entre lo que ha ocurrido y lo que se ha predicho, los valores contra más lejos de la línea roja están más error ha habido, ninguno de los años se puede decir que sea preciso. Por último vamos a ver una gráfica de todos los valores de estos últimos 6 años que nos mostrará también el error de caa muestra pero de una manera más visual.

In [ ]:
plt.scatter(y_val, y_predict)
plt.plot([0, 25], [0, 25], color='red', linewidth=3)

El eje de las X son los valores reales y el eje vertical son los valors predichos, en una perfecta predicción los puntos seguirián la línia roja diagonal, pero este no es el caso y los puntos se encuentran muy dispersos.

## 3.3. Clusterización
En este apartado aplicaremos un algoitmo de clusterización no supervisado, el kmeans, para hacer agrupaciones de los países que más se parecen entre ellos, de esta manera podemos encontrar que países son más parecidos entre ellos y que características son más relevantes de cada uno.

Para esto vamos a transformar el dataset para poder hacer una clusterización según la demografía y otra según la proporción de suicidios de cada grupo de individuos.

### 3.3.1. Clusterización por demografía
Ahora en la función de abajo vamos a crear nuestro nuevo dataset, en el cuál veremos que porcentage de cada grupo de personas hay en cada país por año.

In [ ]:
df_clusters_demografia = pd.DataFrame(columns=['country', 'year', 'population', 'female (%)', '5-14 years',
                                    '15-24 years', '25-34 years', '35-54 years', '55-74 years', '75+ years',
                                    'suicides/100k pop', 'gdp_per_capita ($)'])
df_clusters_demografia['country'] = dataset4['country']
df_clusters_demografia['year'] = dataset4['year']
df_clusters_demografia['population'] = dataset4['population']
for i, fila in dataset4.iterrows():
    if 'female' == fila['sex']:
        df_clusters_demografia.loc[i, 'female (%)'] = fila['population']
    #else:
        #df_clusters_demografia.loc[i, 'female (%)'] = fila['population']
    df_clusters_demografia.loc[i, fila['age']] = fila['population']
    df_clusters_demografia.loc[i, 'suicides/100k pop'] = dataset2.loc[i, 'suicides_no']
    
df_clusters_demografia = df_clusters_demografia.fillna(0)
df_clusters_demografia = df_clusters_demografia.groupby(['country', 'year'], as_index=False).sum()

df_rich = dataset4.groupby(['country', 'year'], as_index=False).mean()
df_clusters_demografia['gdp_per_capita ($)'] = df_rich['gdp_per_capita ($)']
for i, fila in df_clusters_demografia.iterrows():
    df_clusters_demografia.loc[i, 'suicides/100k pop'] = (fila['suicides/100k pop'] / fila['population'])*100000
    df_clusters_demografia.loc[i, 'female (%)'] = (fila['female (%)'] / fila['population'])
    df_clusters_demografia.loc[i, '5-14 years'] = (fila['5-14 years'] / fila['population'])
    df_clusters_demografia.loc[i, '15-24 years'] = (fila['15-24 years'] / fila['population'])
    df_clusters_demografia.loc[i, '25-34 years'] = (fila['25-34 years'] / fila['population'])
    df_clusters_demografia.loc[i, '35-54 years'] = (fila['35-54 years'] / fila['population'])
    df_clusters_demografia.loc[i, '55-74 years'] = (fila['55-74 years'] / fila['population'])
    df_clusters_demografia.loc[i, '75+ years'] = (fila['75+ years'] / fila['population'])
    
scaler = preprocessing.MinMaxScaler()
a, aux = standarize(df_clusters_demografia.values[:,2:], df_clusters_demografia.values[:,2:])
nombres = list(df_clusters_demografia)
indice_fila = 0
for i, fila in df_clusters_demografia.iterrows():
    index = 0
    for index in range(len(nombres)-2):
        df_clusters_demografia.loc[i, nombres[index+2]] = a[indice_fila][index]
        index +=1
    indice_fila += 1
    
df_clusters_demografia.head()

Como vemos tenemos atributos diferentes, pero son simplemente transformaciones de la información que ya teníamos antes, para agilizar hemos estandarizado directamente los datos con el *MinMaxScaler*. Vamos a pasar a explicar cada atributo lo que significa:
- *country*: Es el nombre del país, no requiere estandarización, es nuestro output.
- *year*: Año, no requiere estandarización, vamos a diferenciar los años.
- *population*: Cantidad de población ese año en ese país.
- *female (%)*: Porcentage de mujeres en el país.
- *5-14 years*: Porcentage de población en el rango de edad de 5 a 14 años.
- *15-24 years*: Porcentage de población en el rango de edad de 15 a 14 años.
- *25-34 years*: Porcentage de población en el rango de edad de 25 a 34 años.
- *35-54 years*: Porcentage de población en el rango de edad de 35 a 54 años.
- *55-74 years*: Porcentage de población en el rango de edad de 55 a 74 años.
- *75+ years*: Porcentage de población superior a los 75 años de edad.
- *suicides/100k pop*: Proporción de suicidios por cada 100 mil personas.
- *gdp_per_capita (\$)*: PIB por capita.

Lo siguiente que vamos a hacer es ver un report de este dataframe a ver si tenemos algo relevante que destacar.

In [ ]:
prof = pdp.ProfileReport(df_clusters_demografia)
prof.to_file(output_file='clusters_demografia.html')
prof

No hay nada especialmente llamativo a resaltar,lo mas relevante quizas sea la alta cardinalidad del atributo *country* y que el atributo *suicides/100k pop* contiene también una alta frecuencia de zeros, pero eso es algo que hemos sufrido durante todo el estudio.

La idea ahora es la siguiente, cogeremos un año actual, como 2014 y agruparemos por clusters y luego cogeremos un año lejano, 1995, y veremos si los clusters mantienen las mismas propiedades, en caso afirmativo veremos un patrón constante durante el paso de los años, si por el contrario son diferentes veremos que el paso de los años afecta a la población.

In [ ]:
df_clusters_demografia_2014 = df_clusters_demografia[df_clusters_demografia['year'] == 2014].drop(['country', 'year'], axis=1)
df_clusters_demografia_2014.shape

In [ ]:
nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in nc]
score = [kmeans[i].fit(df_clusters_demografia_2014).score(df_clusters_demografia_2014) for i in range(len(kmeans))]
score
plt.xlabel('Número de clústeres (k)')
plt.ylabel('Suma de los errores cuadráticos')
plt.plot(nc,score)

Podemos ver la elbow curve la cuál nos dice que la mejor k es aquella en la que esta curva comienza a suavizarse, en este caso sería de 10, pero analizando nuestros datos vemos que tenemos pocos atributos, en concreto 12 contando el año y el país que no se van a utilizar para el Kmeans. Así que para que no haya los mismos grupos que atributos vamos a escoger una k de 4.

In [ ]:
kmeans_demografia_2014 = KMeans(n_clusters=4).fit(df_clusters_demografia_2014)
print(kmeans_demografia_2014.cluster_centers_)

Podemos ver los valores de los centroides, ahora los representaremos para que sean más entendibles.

In [ ]:
centroids_demografia_2014 = kmeans_demografia_2014.cluster_centers_
name_centroids = []
for i in range(centroids_demografia_2014.shape[0]):
    name_centroids.append('centroid '+str(i))
df_centroids_demografia_2014 = pd.DataFrame(data=centroids_demografia_2014.transpose(), index=list(df_clusters_demografia_2014), columns=name_centroids)
df_centroids_demografia_2014

In [ ]:
plot_dem_2014 = df_centroids_demografia_2014.plot(kind='bar', figsize=[30,10],
                                            title='Agrupación de paises por la demografía en 2014')


Hemos creado 4 grupos diferentes según la demagrafía en 2014, vamos a pasar a definir cada uno de estos grupos y ver que características tiene cada uno.

- Cluster 0: Es el grupo con mayor ratio de suicidios y son paises con poco nivel económico. Su población es más abundante en las franjas de edad superiores de 34 años, no hay mucha cantidad de población y el porcentage de hombre y mujeres es equilibrado pero con un ligero dominio de la población masculina.

- Cluster 1: Estos paises han pocos suicidios y son los paises mas pobres, su población es abundante y equilibrada en género, las edad más habituales son la franja de 35 a 54 años y lo de 5 a 14 años.

- Cluster 2: Estos son paises con gran indice de suicidios pero son los más ricos, su población es muy abundante y con predominancia entre las edades mayores a 34 años, el genero de la población esta equilibrado.

- Cluster 3: Este grupo lo conforman paises con muy pocos suicidios, muy poca población y paises ricos, con poco porcentage de mujeres y población en la franja de edad joven y media, entre 25 y 54, la población mayor de esta edad es muy escasa en estos paises.

Ahora que ya hemos visto los 4 grupos formados en 2014, vamos a ver que 4 grupos se forman en 1995.

In [ ]:
df_clusters_demografia_1995 = df_clusters_demografia[df_clusters_demografia['year'] == 1995].drop(['country', 'year'], axis=1)
kmeans_demografia_1995 = KMeans(n_clusters=4).fit(df_clusters_demografia_1995)
centroids_demografia_1995 = kmeans_demografia_1995.cluster_centers_
name_centroids = []
for i in range(centroids_demografia_1995.shape[0]):
    name_centroids.append('centroid '+str(i))
df_centroids_demografia_1995 = pd.DataFrame(data=centroids_demografia_1995.transpose(), index=list(df_clusters_demografia_1995), columns=name_centroids)
plot_dem_1995 = df_centroids_demografia_1995.plot(kind='bar', figsize=[30,10],
                                                 title='Agrupación de paises por la demografía en 1995')

Ahora en 1995 han cambiado las clases y tiene otras características diferentes:

- Cluster 0: Este grupo tiene paises con un alto indice de suicidios, tienen bastante riqueza y tiene muy poca población con igualdad en género, son paises con la media de edad entre los 35 y los 74 años.

- Cluster 1: Estos son paises con baja proporción de suicidios pero bastante pobres y con poca población con un ligero debalace a favor de los hombres y es una población muy joven, con dominación de la población joven entre los 5 y los 14 años.

- Cluster 2: Estos son paises tiene un ratio de suicidios alto y con riqueza muy alta y con población muy abundante y equilibrada, la población mayoritaría tiene una edad comprendida entre los 35 y los 55 años.

- Cluster 3: El grupo que conforma estos paises tiene mucha proporción de suicidios y son muy pobres, la población es estandard pero con mucho dominio femenino, es un población bastante equilibrada en edades, los rangos de edades más frecuentes son de 5 a 14 años y entre 53 y 74 años.

Después de ver esto pasaremos a hacer la clusterización teniendo en cuenta el ratio de suicidios de cada grupo de personas por país y año.

### 3.3.2. Clusterización por ratio de suicidios de grupos
Vamos a crear el nuevo dataset el con el que veremos por cada país y año que proporción de suicidios hay de cada grupo de personas.

In [ ]:
df_clusters_rate = pd.DataFrame(columns=['country', 'year', 'population', 'male', 'female', '5-14 years',
                                    '15-24 years', '25-34 years', '35-54 years', '55-74 years', '75+ years',
                                    'suicides/100k pop', 'gdp_per_capita ($)'])
df_clusters_rate['country'] = dataset4['country']
df_clusters_rate['year'] = dataset4['year']
df_clusters_rate['population'] = dataset4['population']
for i, fila in dataset4.iterrows():
    if 'male' == fila['sex']:
        df_clusters_rate.loc[i, 'male'] = dataset2.loc[i, 'suicides_no']
    else:
        df_clusters_rate.loc[i, 'female'] = dataset2.loc[i, 'suicides_no']
    df_clusters_rate.loc[i, fila['age']] = dataset2.loc[i, 'suicides_no']
    df_clusters_rate.loc[i, 'suicides/100k pop'] = dataset2.loc[i, 'suicides_no']
    
df_clusters_rate = df_clusters_rate.fillna(0)
df_clusters_rate = df_clusters_rate.groupby(['country', 'year'], as_index=False).sum()

df_rich = dataset4.groupby(['country', 'year'], as_index=False).mean()
df_clusters_rate['gdp_per_capita ($)'] = df_rich['gdp_per_capita ($)']
for i, fila in df_clusters_rate.iterrows():
    df_clusters_rate.loc[i, 'suicides/100k pop'] = (fila['suicides/100k pop'] / fila['population'])*100000
    df_clusters_rate.loc[i, 'male'] = (fila['male'] / fila['population'])*100000
    df_clusters_rate.loc[i, 'female'] = (fila['female'] / fila['population'])*100000
    df_clusters_rate.loc[i, '5-14 years'] = (fila['5-14 years'] / fila['population'])*100000
    df_clusters_rate.loc[i, '15-24 years'] = (fila['15-24 years'] / fila['population'])*100000
    df_clusters_rate.loc[i, '25-34 years'] = (fila['25-34 years'] / fila['population'])*100000
    df_clusters_rate.loc[i, '35-54 years'] = (fila['35-54 years'] / fila['population'])*100000
    df_clusters_rate.loc[i, '55-74 years'] = (fila['55-74 years'] / fila['population'])*100000
    df_clusters_rate.loc[i, '75+ years'] = (fila['75+ years'] / fila['population'])*100000
    
scaler = preprocessing.MinMaxScaler()
a, aux = standarize(df_clusters_rate.values[:,2:], df_clusters_rate.values[:,2:])
nombres = list(df_clusters_rate)
print(type(a))
indice_fila = 0
for i, fila in df_clusters_rate.iterrows():
    index = 0
    for index in range(len(nombres)-2):
        df_clusters_rate.loc[i, nombres[index+2]] = a[indice_fila][index]
        index +=1
    indice_fila += 1
    
df_clusters_rate

Igualmente que antes explicaremos los atributos nuevos que se han creado:
- *country*: Es el nombre del país, no requiere estandarización, es nuestro output.
- *year*: Año, no requiere estandarización, vamos a diferenciar los años.
- *population*: Cantidad de población ese año en ese país.
- *female*: Proporción por 100 mil personas de mujeres que se han suicidado.
- *male*: Proporción por 100 mil personas de hombres que se han suicidado.
- *5-14 years*:  Proporción por 100 mil personas en el rango de edad de 5 a 14 años.
- *15-24 years*: Proporción por 100 mil personas en el rango de edad de 15 a 14 años.
- *25-34 years*: Proporción por 100 mil personas en el rango de edad de 25 a 34 años.
- *35-54 years*: Proporción por 100 mil personas en el rango de edad de 35 a 54 años.
- *55-74 years*: Proporción por 100 mil personas en el rango de edad de 55 a 74 años.
- *75+ years*: Proporción por 100 mil personas con edad superior a los 75 años.
- *suicides/100k pop*: Proporción de suicidios por cada 100 mil personas.
- *gdp_per_capita (\$)*: PIB por capita.

Lo siguiente que vamos a hacer es ver un report de este dataframe a ver si tenemos algo relevante que destacar.

In [ ]:
prof = pdp.ProfileReport(df_clusters_rate)
prof.to_file(output_file='clusters_rate.html')
prof

El report nos avisa de que todas las proporciones de suicidios contiene muchos 0, por ejemplo pordemos ver que para gente entre 5 y 14 años tenemos aproximadamente un cuarto de los datos que son 0, lo que significa que en esa franja de edad hay muchos años en algunos paises que no se suicida nadie. Otra cosa curiosa que vemos es que la proporcion total de suicidios por cada 100 mil personas esta muy correlacionado con la proporción de suicidios entre las mujeres y el rango de suicidios de población entre 35 y 54 años. Luego también nos avisa como siempre sobre la alta cardinalidad del atributo con el nombre de los países.

La idea es con esto agrupar en grupos donda cada grupo contendrá unas características diferenciadoras de los demás, primero miraremos un año más actual como 2014 y un más antiguo, 1995. Por el número de muestras que contiene son los años más separados y que contiene un alto número de muestras.

In [ ]:
df_clusters_rate_2014 = df_clusters_rate[df_clusters_rate['year'] == 2014].drop(['country', 'year'], axis=1)
df_clusters_rate_2014.shape

In [ ]:
nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in nc]
score = [kmeans[i].fit(df_clusters_rate_2014).score(df_clusters_rate_2014) for i in range(len(kmeans))]
score
plt.xlabel('Número de clústeres (k)')
plt.ylabel('Suma de los errores cuadráticos')
plt.plot(nc,score)

Aquí podemos ver la elbow curve qye no indica que la mejor k se situaría alrededos de un valor aproximado de 5, por lo que se harán agrupaciones de 5 grupos diferentes.

In [ ]:
kmeans_rate_204 = KMeans(n_clusters=5).fit(df_clusters_rate_2014)
print(kmeans_rate_204.cluster_centers_)

Bueno aquí tenemos los centroides de cada cluster, pero esto no es muy visual y es difícil de interpretar, por eso vamos a representarlos de manera gráfica y a partir de aquí extraeremos las conclusiones pertinentes.

In [ ]:
centroids_rate_2014 = kmeans_rate_204.cluster_centers_
name_centroids = []
for i in range(centroids_rate_2014.shape[0]):
    name_centroids.append('centroid '+str(i))
df_centroids_rate_2014 = pd.DataFrame(data=centroids_rate_2014.transpose(), index=list(df_clusters_rate_2014), columns=name_centroids)
df_centroids_rate_2014

In [ ]:
plot_rate_2014 = df_centroids_rate_2014.plot(kind='bar', figsize=[30,10],
                                            title ='Agrupación de paises por la proporción de suicidios en 2014')

Vamos a difentificar los 5 grupos formados por los ratios de suicidios de cada tipo de persona:

- Cluster 0: El primer grupo lo conforman países con una alta proporción de suicidios, son países pobres y con un valor bajo de población. Los hombres se suicidan en proporción mucho más alta que los hombres y el rango de edad más propenso es el de 55 a 74 años, siendo los más pequeños los que menos se suicidan.

- Cluster 1: Estos países son muy ricos pero no tiene un gran índice de suicidio, la población es grande y son también los hombres los que más se suicidan, el rango más alto en suicidios es de 55 a 74 años y los más pequeños apenas se suicidan.

- Cluster 2: Son un grupo con un índice de suicidios relativamente bajo, son pobres y con bastante población, domina el género masculino como el más propenso y siendo los frecuente la población comprendida entre los 25 a los 54.

- Cluster 3: Son el grupo con mayor indice de suicidios y son ligeramente mas ricos que el grupo anterior. Su población es abundante y con propensión a los suicidios de hombre entre 55 y 74 años.

- Cluster 4: Este es un grupo con muy bajo índice de suicidios y con buen nivel económico, la población es escasa y son los hombre quien se suicidan en mayor proporción siendo el ranfo de 25 a 34 los que más.

Igual que antes vamos a comparar si el paso de los años mantiene una tendencia en los grupos o por si lo contrario las conclusiones que se extraen de los años más antiguos no aplican en los tiempos modernos.

In [ ]:
df_clusters_rate_1995 = df_clusters_rate[df_clusters_rate['year'] == 1995].drop(['country', 'year'], axis=1)
kmeans_rate_1995 = KMeans(n_clusters=5).fit(df_clusters_rate_1995)
centroids_rate_1995 = kmeans_rate_1995.cluster_centers_
name_centroids = []
for i in range(centroids_rate_1995.shape[0]):
    name_centroids.append('centroid '+str(i))
df_centroids_rate_1995 = pd.DataFrame(data=centroids_rate_1995.transpose(), index=list(df_clusters_rate_1995), columns=name_centroids)
plot_rate_1995 = df_centroids_rate_1995.plot(kind='bar', figsize=[30,10],
                                            title ='Agrupación de paises por la proporción de suicidios en 1995')

Ahora los grupos han variado así que explicaremos un poco las características de cada grupo:
- Cluster 0: Este grupo tiene los suicidios generales relativamente bajos y son un país rico con mucha población. Es el género masculino en el rango de los 35 a los 54 el más predispuesto.

- Cluster 1: Estos países son muy ricos pero con el mayor índice de suicidios, la población es grande y son también los hombres los que más se suicidan, el rango más alto en suicidios es de 55 a 74 años y los más pequeños apenas se suicidan.

- Cluster 2: Son un grupo con un índice de suicidios bajo y son pobres y con poca población, domina el género masculino como el más propenso y siendo los frecuente la población comprendida entre los 25 a los 34.

- Cluster 3: Son el grupo con indice de suicidios  muy grande y son muy pobres. Su población es abundante y con propensión a los suicidios de mujeres mayores de 75 y entre 15 y 24 años.

- Cluster 4: Este es un grupo con alto índice de suicidios y con muy bajo nivel económico, la población es muy escasa y son los hombre quien se suicidan en mayor proporción siendo el ranfo de 25 a 34 los que más.

Una vez visto esto lo que vamos a hacer es analizar lo que hemos ido haciendo en el apartado de clusterización y extraer todas las conclusiones que se pueda.

### 3.3.3 Conclusiones
Después de haber terminado con esto vamos extraer toda la información que se pueda de la clusterización que hemos realizado en los apartados anteriores.

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2)
df_centroids_demografia_2014.plot(kind='bar', figsize=[35,30], ax = axs[0][1],
                                        title ='Agrupación de paises por la demografía en 2014')
df_centroids_demografia_1995.plot(kind='bar', figsize=[35,30], ax = axs[0][0],
                                        title ='Agrupación de paises por la demografía en 1995')
df_centroids_rate_2014.plot(kind='bar', figsize=[35,30], ax = axs[1][1],
                                        title ='Agrupación de paises por la proporción de suicidios en 2014')
df_centroids_rate_1995.plot(kind='bar', figsize=[35,30], ax = axs[1][0],
                                        title ='Agrupación de paises por la proporción de suicidios en 1995')

Como se puede apreciar a lo largo de los años hay pequeños cambios que hacen que no se puedan agrupar los grupos igual para todos los años, sin embargo hay tendencias que perdurán y nos da información de lo que ocurre.

En lo que respecta a la demografía vemos que el rango de edad es lo más importante a la hora de determinar la proporción de suicidios en el país, si son una población joven tienden a gaber menos suicidios que en poblaciones con edades más avanzadas.

Cuando hemos agrupado por ratio de suicidios de cada tipo de persona hemos visto que hay una tendencia clara, los hombres entre que comprenden edades entre los 35 y los 74 son los más propensoso al suicidio

Con esto podemos ver que, a pesar de que a simple vista pueda parecer que el dinero es un factor determinante para ver si una población tiende o no al suicidio esto no es cierto. La población tampoco es algo que nos aporte demsiada información. Concluimos que sabiendo la edad de las personas que viven en estos paises y el genero podemos determinar si será un país con muchos o pocos suicidios y que seguramente la mayor porte vengan del género masculino de mediana edad.